<a href="https://colab.research.google.com/github/gauravrana100/practice_transformers/blob/main/fine_tuning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g5s5p0zuu5ni_w4ekunbcRSuO9f7OApnr-bRa0kLBV3euKF3r_Lr0Y
Mounted at /content/drive


In [ ]:
# ! pip install datasets transformers

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Curated DLD Data Foundation Apr 1 V1.0.xlsx to Curated DLD Data Foundation Apr 1 V1.0.xlsx


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
!ls /content/drive/MyDrive/Transformer-transfer-learning/masked-language-modelling

fine_tuning_model.ipynb


In [ ]:
# !mv "Curated DLD Data Foundation Apr 1 V1.0.xlsx" /content/drive/MyDrive/Transformer-transfer-learning/masked-language-modelling

In [ ]:
os.chdir("/content/drive/MyDrive/Transformer-transfer-learning/masked-language-modelling")

In [ ]:
os.listdir()

['fine_tuning_model.ipynb', 'Curated DLD Data Foundation Apr 1 V1.0.xlsx']

In [ ]:
import pandas as pd

In [164]:
df = pd.read_excel("Curated DLD Data Foundation Apr 1 V1.0.xlsx")

In [165]:
df.shape

(58380, 45)

In [166]:
df.columns

Index(['S.No', 'Industry', 'Sub Industry', 'Solution', 'GSI', 'GSI Role',
       'GSI Solution Design Rights', 'GSI Transaction Rights',
       'GSI Role Description', 'CU', 'Layers', 'Entity', 'CU Role',
       'CU Solution Design Rights', 'CU Transaction Rights',
       'CU Role Description', 'Entity Role', 'Entity Solution Design Rights',
       'Entity Transaction Rights', 'Entity Role Description', 'Agent',
       'Attribute', 'Attribute Role', 'Attribute Solution Design Rights',
       'Attribute Transaction Rights', 'Attribute Role Description',
       'isReserved', 'reservedCUType', 'ReservedCU Membership Criteria',
       'AttributeDataType', 'AttributeDataSize', 'AttributeUIType',
       'DCDTarget', 'DCDSource', 'DCDFormula', 'DCDEvent', 'GSI Sequence No',
       'CU Sequence No', 'CU Type', 'CU Logic', 'Recursive Constraint',
       'Estimated Time', 'version', 'masterId', 'GSI ID'],
      dtype='object')

In [167]:
df = df[['GSI', 'CU', 'Entity', 'Attribute']]
df.drop_duplicates(keep='first', inplace=True)

In [168]:
df.shape

(48657, 4)

In [169]:
df.reset_index(drop=True, inplace=True)

In [170]:
df.head()

,GSI,CU,Entity,Attribute
0,Order Pizza,Register Customer,Customer,Customer Name
1,Order Pizza,Register Customer,Customer,Customer Address
2,Order Pizza,Register Customer,Customer,Customer Mobile Number
3,Order Pizza,Register Customer,Customer,Customer Email ID
4,Order Pizza,Register Customer,Customer,Customer ID


# Prepare data for fine tuning

In [171]:
df.loc[0]

GSI                Order Pizza
CU           Register Customer
Entity                Customer
Attribute        Customer Name
Name: 0, dtype: object

In [172]:
'For the process to ' + df.loc[0]['GSI'] + ' we need to ' + df.loc[0]['CU'] + 'for entity ' + df.loc[0]['Entity'] + ' for attribute ' + df.loc[0]['Attribute']

'For the process to Order Pizza we need to Register Customerfor entity Customer for attribute Customer Name'

In [173]:
tmp = []
for i in range(df.shape[0]):
  tmp.append('For the process to ' + df.loc[i]['GSI'] + ' we need to ' + df.loc[i]['CU'] + 'for entity ' + df.loc[i]['Entity'] + ' for attribute ' + df.loc[i]['Attribute'])

In [174]:
tmp[:5]

['For the process to Order Pizza we need to Register Customerfor entity Customer for attribute Customer Name',
 'For the process to Order Pizza we need to Register Customerfor entity Customer for attribute Customer Address',
 'For the process to Order Pizza we need to Register Customerfor entity Customer for attribute Customer Mobile Number',
 'For the process to Order Pizza we need to Register Customerfor entity Customer for attribute Customer Email ID',
 'For the process to Order Pizza we need to Register Customerfor entity Customer for attribute Customer ID']

In [175]:
len(tmp)

48657

In [176]:
df['text'] = tmp
df.drop_duplicates(inplace=True)

In [177]:
df.shape

(48657, 5)

In [178]:
df.reset_index(drop=True, inplace=True)

In [179]:
from sklearn.model_selection import train_test_split

In [224]:
train, test = train_test_split(df[['text']], test_size=0.2)

In [225]:
train.shape, test.shape

((38925, 1), (9732, 1))

In [226]:
train.head()

,text
23343,For the process to Perform Business to Custome...
40921,For the process to Perform Rating Process we n...
11031,For the process to Perform Leader Onboarding w...
44205,For the process to Review Job Posting Info we ...
32938,For the process to Perform Lead Management we ...


In [227]:
need_dataset = {
    'train': train['text'].to_list(),
    'test': test['text'].to_list()
}

# need_dataset_df = pd.DataFrame(need_dataset)

In [228]:
need_dataset.keys()

dict_keys(['train', 'test'])

### Preparing the dataset

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by [MASK]) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).



In [229]:
type(need_dataset['train'])

list

In [230]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [231]:
train_dataset

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 38925
})

### Modelling

In [232]:
from transformers import AutoTokenizer

In [233]:
model_checkpoint = "distilroberta-base"

In [234]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [235]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
train_tokenized_datasets = train_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text", "__index_level_0__"])
test_tokenized_datasets = test_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text", "__index_level_0__"])

In [236]:
train_tokenized_datasets

Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 38925
})

In [237]:
train_lm_datasets = train_tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

test_lm_datasets = test_tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [238]:
train_lm_datasets

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 8778
})

In [256]:
train_lm_datasets.data

ConcatenationTable
attention_mask: list<item: int8>
  child 0, item: int8
input_ids: list<item: int32>
  child 0, item: int32
labels: list<item: int64>
  child 0, item: int64

First we use a model suitable for masked LM:

In [239]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

And second, we use a special data_collator. The data_collator is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the data_collator, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a DataCollatorForLanguageModeling. We can adjust the probability of the masking:

In [240]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to Trainer and begin training:

In [241]:
from transformers import Trainer, TrainingArguments

In [257]:
training_args = TrainingArguments(
    "test-mlm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [258]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_lm_datasets,
    eval_dataset=test_lm_datasets,
    data_collator=data_collator,
)

In [259]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.923600,0.950389
2,0.859400,0.860817
3,0.898500,0.822980


TrainOutput(global_step=3294, training_loss=0.8997055544734508, metrics={'train_runtime': 1080.5243, 'train_samples_per_second': 3.049, 'total_flos': 148048351907328.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -757882880, 'train_mem_gpu_alloc_delta': 663909888, 'train_mem_cpu_peaked_delta': 758247424, 'train_mem_gpu_peaked_delta': 1276403712})

### Save Model

In [260]:
model.save_pretrained("./dld_data_model")

### Load model

In [261]:
pytorch_model = AutoModelForMaskedLM.from_pretrained('./dld_data_model')

In [1]:
# !pip install transformers

In [262]:
from tansformers import pipeline

ModuleNotFoundError: ignored